This script will make a train/val set up to 100:1 ratio negative to positive, and test set of 10% all data with true distrubtion (4000:1) of negative to positive

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import uuid
import numpy as np
import pandas as pd

import pyarrow as pa
import pyarrow.dataset as ds
import pyarrow.parquet as pq

# ================== CONFIG ==================
SRC_PARQUET_DIR = "/explore/nobackup/people/spotter5/clelland_fire_ml/parquet_cems_with_fraction_dataset"

OUT_ROOT    = "/explore/nobackup/people/spotter5/clelland_fire_ml"
TRAIN100_DIR = os.path.join(OUT_ROOT, "parquet_cems_trainval_100x")
TEST_DIR     = os.path.join(OUT_ROOT, "parquet_cems_test_true_10pct")

os.makedirs(TRAIN100_DIR, exist_ok=True)
os.makedirs(TEST_DIR, exist_ok=True)

# True global neg:pos ratio you measured:
GLOBAL_NEG_POS_RATIO = 4169.10   # from your previous result

TEST_FRACTION   = 0.10           # 10% test set (true ratio)
TARGET_NEG_POS  = 100.0          # target ~100:1 neg:pos in Train/Val

# Approximate probability of keeping a negative in TrainVal
NEG_KEEP_PROB = min(1.0, TARGET_NEG_POS / GLOBAL_NEG_POS_RATIO)

BATCH_SIZE   = 500_000
RANDOM_STATE = 42

np.random.seed(RANDOM_STATE)

# ================== HELPERS ==================
def write_parquet_chunk(df: pd.DataFrame, root: str, prefix: str):
    """Write a chunk as a standalone Parquet file with a unique name."""
    if df.empty:
        return
    table = pa.Table.from_pandas(df, preserve_index=False)
    fname = f"{prefix}_part_{uuid.uuid4().hex}.parquet"
    fpath = os.path.join(root, fname)
    pq.write_table(table, fpath)

# ================== MAIN ==================
print(f"Scanning source Parquet dataset: {SRC_PARQUET_DIR}")
dataset = ds.dataset(SRC_PARQUET_DIR, format="parquet")

total_rows = 0
test_pos = test_neg = 0
train_pos = train_neg = 0

print(f"\nUsing TEST_FRACTION    = {TEST_FRACTION:.2f}")
print(f"Target TrainVal neg:pos ≈ {TARGET_NEG_POS:.1f}:1")
print(f"Using NEG_KEEP_PROB   ≈ {NEG_KEEP_PROB:.6f} for negatives in TrainVal\n")

for batch_idx, batch in enumerate(dataset.to_batches(batch_size=BATCH_SIZE), start=1):
    if batch.num_rows == 0:
        continue

    df = batch.to_pandas()

    if "fraction" not in df.columns:
        raise ValueError("Column 'fraction' missing in dataset!")

    frac = df["fraction"].to_numpy(dtype="float32")
    burned = frac > 0.5  # True for positives

    n_rows = len(df)
    total_rows += n_rows

    # ----------------- TEST ASSIGNMENT (TRUE RATIO) -----------------
    # Randomly assign ~10% of all rows to test, regardless of class
    u_test = np.random.rand(n_rows)
    is_test = (u_test < TEST_FRACTION)
    not_test = ~is_test

    # TEST set (keeps true ~4169:1 distribution)
    df_test = df[is_test].copy()
    if not df_test.empty:
        df_test["burned"] = burned[is_test].astype("uint8")
        write_parquet_chunk(df_test, TEST_DIR, prefix="test")

        test_pos += burned[is_test].sum()
        test_neg += (~burned[is_test]).sum()

    # ----------------- TRAIN/VAL (TARGET ~100:1) -----------------
    # all positives in TrainVal (from the 90% non-test pool)
    pos_train_mask = not_test & burned

    # negatives in TrainVal, subsampled with NEG_KEEP_PROB
    neg_train_cand_mask = not_test & (~burned)
    u_neg = np.random.rand(n_rows)
    neg_train_mask = neg_train_cand_mask & (u_neg < NEG_KEEP_PROB)

    train_mask = pos_train_mask | neg_train_mask

    df_train = df[train_mask].copy()
    if not df_train.empty:
        df_train["burned"] = burned[train_mask].astype("uint8")
        write_parquet_chunk(df_train, TRAIN100_DIR, prefix="train")

        train_pos += burned[train_mask].sum()
        train_neg += (~burned[train_mask]).sum()

    if batch_idx % 50 == 0:
        print(f"Processed {batch_idx} batches, total_rows so far: {total_rows:,}")

# ================== SUMMARY ==================
print("\n================= SAMPLING SUMMARY =================")
print(f"Total rows scanned              : {total_rows:,}")

print("\n--- TEST SET (10% true distribution) ---")
print(f"Test positives (burned=1)       : {int(test_pos):,}")
print(f"Test negatives (burned=0)       : {int(test_neg):,}")
if test_pos > 0:
    test_neg_pos_ratio = test_neg / test_pos
    print(f"Neg:Pos in TEST                 : {test_neg_pos_ratio:.2f}:1")
    print(f"Percent positives in TEST       : {100 * test_pos / (test_pos + test_neg):.6f} %")

print("\n--- TRAIN/VAL SET (~100:1 target) ---")
print(f"TrainVal positives (burned=1)   : {int(train_pos):,}")
print(f"TrainVal negatives (burned=0)   : {int(train_neg):,}")
if train_pos > 0:
    train_neg_pos_ratio = train_neg / train_pos
    print(f"Neg:Pos in TRAIN/VAL            : {train_neg_pos_ratio:.2f}:1")
    print(f"Percent positives in TRAIN/VAL  : {100 * train_pos / (train_pos + train_neg):.6f} %")

print("\nTrainVal dataset written to:")
print(f"  {TRAIN100_DIR}")
print("Test dataset written to:")
print(f"  {TEST_DIR}")
print("====================================================\n")


Scanning source Parquet dataset: /explore/nobackup/people/spotter5/clelland_fire_ml/parquet_cems_with_fraction_dataset

Using TEST_FRACTION    = 0.10
Target TrainVal neg:pos ≈ 100.0:1
Using NEG_KEEP_PROB   ≈ 0.023986 for negatives in TrainVal

Processed 50 batches, total_rows so far: 1,204,422
Processed 100 batches, total_rows so far: 2,517,301
Processed 150 batches, total_rows so far: 3,880,345
Processed 200 batches, total_rows so far: 5,464,567
Processed 250 batches, total_rows so far: 6,900,785
Processed 300 batches, total_rows so far: 8,280,925
Processed 350 batches, total_rows so far: 9,686,325
Processed 400 batches, total_rows so far: 10,988,053
Processed 450 batches, total_rows so far: 12,426,585
Processed 500 batches, total_rows so far: 13,864,504
Processed 550 batches, total_rows so far: 15,102,539
Processed 600 batches, total_rows so far: 16,392,297
Processed 650 batches, total_rows so far: 17,709,041
Processed 700 batches, total_rows so far: 19,077,236
Processed 750 batches,